In [119]:
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from requests_html import HTML

import re
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

In [125]:
m = Mystem()
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
count_vect = CountVectorizer(stop_words=stop_words, ngram_range=(2, 2))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikita.Chernikov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [138]:
def lemmatize(lst):
    for n in range(len(lst)):
        lst[n] = m.lemmatize(lst[n])
        lst[n] = lst[n][:-1]
    return lst

In [35]:
def clear_text(text):
    text = re.sub(r'[^а-яА-ЯёЁa-zA-Z ]', ' ', text)
    text = text.split()
    return ' '.join(text)

In [75]:
global_counter_modify_jobs = 0

In [76]:
#функция обработки названий работ из резюме 

def hh_job_preparation(string):
    global global_counter_modify_jobs

    result = string.lower()

    temp_string = result

    result = result.replace("зам. ", "заместитель ")
    result = result.replace("нач. ", "начальник ")
    result = result.replace("рук. ", "руководитель ")
    result = result.replace("ген. ", "генеральный ")
    result = result.replace("гендиректор", "генеральный директор")
    result = result.replace("–", "-")
    result = result.replace(" - ", "-")

    if (result.find("(")>0 and result.find(")")>0):
        temp = ""
        not_closed = False
        for i in range(len(result)):
            if result[i] == "(":
                not_closed = True
            elif result[i] == ")":
                not_closed = False
            else:
                if not not_closed:
                    temp += result[i]
        result = temp

    if (result.find("«")>0 and result.find("»")>0):
        temp = ""
        not_closed = False
        for i in range(len(result)):
            if result[i] == "«":
                not_closed = True
            elif result[i] == "»":
                not_closed = False
            else:
                if not not_closed:
                    temp += result[i]
        result = temp

    if (result.find(",") > 0) :
        result=result.partition(',')[0]

    if (result.find(";") > 0) :
        result=result.partition(';')[0]

    if (result.find(".") > 0) :
        result=result.partition('.')[0]

    if (result.find("/") > 0) :
        result=result.partition('/')[0]

    if (result.find(" по ") > 0) :
        result=result.partition(' по ')[0]


    if (result.find("начальник ") > -1 or result.find("руководитель ") > -1):
        if (result.find("отдела") > -1):
            result = "начальник отдела"
        elif (result.find("службы") > -1):
            result = "начальник службы"
        elif (result.find("управления") > -1):
            result = "начальник управления"
        elif (result.find("подразделения") > -1):
            result = "начальник подразделения"
        elif (result.find("группы") > -1):
            result = "начальник группы"

    if (result.find("заместитель начальника") > -1 or result.find("заместитель руководителя ") > -1):
        if (result.find("отдела") > -1):
            result = "заместитель начальника отдела"
        elif (result.find("службы") > -1):
            result = "заместитель начальника службы"
        elif (result.find("управления") > -1):
            result = "заместитель начальника управления"
        elif (result.find("подразделения") > -1):
            result = "заместитель начальника подразделения"
        elif (result.find("группы") > -1):
            result = "заместитель начальника группы"

    if (result.find("developer") > -1 or result.find("programmer") > -1 ):
        result="программист"

    if (result.find("генеральный директор") > -1 or result.find("programmer") > -1 ):
        result = "генеральный директор"

    result = result.strip()

    if (result != temp_string):
        global_counter_modify_jobs += 1


    return(result)

In [96]:
#функция обработки временного периода из резюме с hh
def hh_time_preparation(string):
    # Вариант 1 - Долго работал
    # ['Март', '1986', '—', 'Май', '1989', '3', 'года', '3', 'месяца']
    # Вариант 2 - Круглые года
    # ['Февраль', '2019', '—', 'Январь', '2022', '3', 'года']
    # Вариант 3 - Круглые месяцы
    # ['Май', '2003', '—', 'Август', '2003', '4', 'месяца']
    # Вариант 4 - долго работал и по настоящее время
    # ['Июнь', '2020', '—', 'по', 'настоящее', 'время', '2', 'года', '8', 'месяцев']
    # Вариант 5 - Круглые года и по настоящее время
    # ['Июнь', '2020', '—', 'по', 'настоящее', 'время', '2', 'года']
    # Вариант 6 - Круглые месяцы и по настоящее время
    # ['Май', '2022', '—', 'по', 'настоящее', 'время', '9', 'месяцев']
    word_array = string.split()
    months = 0
    if ((word_array[4] == "настоящее") or (word_array[3] == "currently")): 
        months = 10000
    elif (len(word_array) == 9): 
        months = 12 * int(word_array[5]) + int(word_array[7])
    elif word_array[6] in ['месяца', 'месяцев', 'months', 'month']: 
        months = int(word_array[5])
    elif word_array[6] in ['года', 'лет', 'год', 'years', 'year']: 
        months = 12 * int(word_array[5])
    else: 
        print("Некорректное резюме, проблема со временем в должности: "+ str(word_array))
    return (int(months))

In [149]:
data = []
client_id = 1

In [150]:
def hh_resume_online_parser(input_file):
    global data, client_id
    
    resume = HTML(html=input_file)

    time = []
    job = []
    des = []

    resume_experience = resume.find('div[data-qa=resume-block-experience]')
    resume_experience_len = len(resume_experience)
    if (resume_experience_len == 1):
        resume_times = resume_experience[0].find('.bloko-column.bloko-column_xs-4.bloko-column_s-2.bloko-column_m-2.bloko-column_l-2')
        resume_jobs = resume_experience[0].find('div[data-qa=resume-block-experience-position]')
        resume_des = resume_experience[0].find('div[data-qa=resume-block-experience-description]')
        for i in range(len(resume_jobs)):  # смотрим места работы
            time.append(hh_time_preparation(resume_times[i].text))
            job.append(hh_job_preparation(resume_jobs[i].text))
            des.append(resume_des[i].text)

    elif (resume_experience_len == 0):
        print("Не удалось открыть резюме")
        return 0
    
    else:
        print("Некорректное резюме, количество resume-block-experience = " + str(resume_experience_len))
        return 0

    # des = lemmatize(des)
    
    prof_id = 0
    for n in range(len(job)):
        data.append([client_id, prof_id, job[n], time[n], des[n]])
        prof_id += 1
    client_id += 1

In [ ]:
n_gramm = count_vect.fit_transform(corpus)

In [99]:
files = os.listdir('data\\resumes')

In [151]:
for name in files:
    with open(f'data\\resumes\\{name}', encoding='utf-8') as f:
    # soup = BeautifulSoup(f, 'html.parser')
        file = f.read()
        hh_resume_online_parser(file)

Некорректное резюме, проблема со временем в должности: ['Апрель', '2001', '—', 'Апрель', '2001', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Сентябрь', '2015', '—', 'Сентябрь', '2015', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Октябрь', '2020', '—', 'Октябрь', '2020', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Декабрь', '2019', '—', 'Декабрь', '2019', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Январь', '2015', '—', 'Январь', '2015', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Июль', '2013', '—', 'Июль', '2013', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Май', '2019', '—', 'Май', '2019', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Декабрь', '2012', '—', 'Декабрь', '2012', '1', 'месяц']
Некорректное резюме, проблема со временем в должности: ['Январь', '2008', '—', 'Январь', '2008', '1', 'месяц']
Н

In [153]:
df = pd.DataFrame(data, columns=['client_id', 'prof_id', 'job', 'months', 'description'])

In [155]:
df.to_csv('data.csv')

In [154]:
df

,client_id,prof_id,job,months,description
0,1,0,агроном,10000,"Проверка качества сбора огурца, Работа с перво..."
1,2,0,агроном-консультант,10000,"Консультация,менеджмент, продажа минеральных у..."
2,2,1,администратор сельскохозяйственного кооператив...,36,Обязанности исполняющего директора
3,2,2,начальник отдела,47,Проверка семенного материла по с/х предприятия...
4,2,3,электрик подстанционных сетей,14,Ремонт обслуживание электроподстанций
...,...,...,...,...,...
4330,999,2,консультант приемной комиссии,3,Консультирование абитуриентов
4331,999,3,стажер,3,"Проведение экспериментов, написание отчетов, с..."
4332,1000,0,агроном,28,Выращивание с/х культур
4333,1001,0,ст инженер,10000,"Организация работы, эксплуатация , техническое..."
